In [13]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

import numpy as np

In [14]:
# def extract_salary(soup):
#     salaries = []
#     for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
#         try:
#             salaries.append(div.find('nobr').text)
#         except:
#             try:
#                 div_two = div.find(name = 'div', attrs = {'class':'sjcl'})
#                 div_three = div_two.find('div')
#                 salaries.append(div_three.text.strip())
# #                 print(div_three)
#             except:
#                 salaries.append('Nothing_found')    
#     return(salaries)

def extract_salary(soup):
    salary = []
    for div in soup.find_all(name='div', attrs = {'class':'row'}):
        if div.find(name = 'span', attrs = {'class':"salaryText"}):
            span = div.find(name = 'span', attrs = {'class':"salaryText"})
            salary.append(span.text)
        else:
            salary.append(np.NaN)
    return salary

def extract_summary(soup):
    summaries = []
    divs = soup.find_all('div', attrs = {'class':'summary'})
#     print(divs)
    for i, div in enumerate(divs):
#         print(i, div.text)
        summaries.append(div.text.strip())
    return(summaries)

def extract_location(soup):
    locations = []
    spans = soup.find_all('span', attrs={'class':'location'})
    for span in spans:
        locations.append(span.text)
    return(locations)

def extract_company(soup):
    companies = []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        company = div.find_all(name = 'span', attrs = {'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())
    return(companies)

def extract_job_title(soup):
    jobs = []
    for div in soup.find_all(name='div', attrs = {'class':'row'}):
        for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
            jobs.append(a['title'])
    return(jobs)

def extract_post_date(soup):
    dates = []
    for div in soup.find_all(name='div', attrs = {'class':'row'}):
        for span in div.find_all(name = 'span', attrs = {'class':"date"}):
            dates.append(span.text)
    return dates

In [15]:
# URL = 'https://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l=New+York&start=10'
# URL = 'https://www.indeed.co.uk/jobs?q=quantitative+analyst+%C2%A340,000&l=London'
URL = 'https://www.indeed.co.uk/jobs?q=Finance&l=London&radius=100' # finance job in London
#conducting a request of the stated URL above:
page = requests.get(URL)
#specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
soup = BeautifulSoup(page.text, 'html.parser')
#printing soup in a more structured tree format that makes for easier reading
# print(soup.prettify())

In [18]:
URL = 'https://www.indeed.co.uk/jobs?q=quantitative+analyst+%C2%A340,000&l=London'
page = requests.get(URL)
soup = BeautifulSoup(page.text, 'html.parser')

title = extract_job_title(soup)

company = extract_company(soup)

ls = extract_location(soup)

summary = extract_summary(soup)
    
salary = extract_salary(soup)

# date = extract_post_date(soup)

# df1 = zip(company, title, ls, salary, summary)
# df = pd.DataFrame(df1, columns = ['company', "title", 'location', 'salary', 'summary'])
# df.head()

In [24]:
company

['Selby Jennings',
 'Spencer Clarke Group',
 'Selby Jennings',
 'IHS Markit',
 'NHS England and NHS Improvement',
 'Google',
 'Deutsche Bank',
 'Ipreo',
 'CITI',
 'CITI',
 'CITI',
 'Deutsche Bank',
 'UK Government - Office for National Statistics']

In [27]:
dict(zip(company, title))

{'Selby Jennings': 'FO Quantitative Analyst - Pricing',
 'Spencer Clarke Group': 'Behavioral Insight Analyst',
 'IHS Markit': 'Quantitative Analyst, iBoxx Indices',
 'NHS England and NHS Improvement': 'Analyst',
 'Google': 'Financial Analyst, Hardware',
 'Deutsche Bank': 'Quantitative Analyst',
 'Ipreo': 'Quantitative Analyst, iBoxx Indices',
 'CITI': 'XVA Quantitative Analyst',
 'UK Government - Office for National Statistics': 'Methods, Data and Research Analyst'}

In [30]:
# df1 = dict(zip(company, title))
df1 = zip(company, title)
pd.DataFrame(df1)

TypeError: data argument can't be an iterator

In [11]:
## fetching metadata
max_jobs_to_fetch = 1000
columns = ['location', 'company_name', 'job_title', 'summary']
# columns = ['company', 'title']
df = pd.DataFrame(columns = columns)

for start in range(0, max_jobs_to_fetch, 10):
    try:
        url = 'https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start='
        page = requests.get(url + str(start))
        print('retriving url: ', (url + str(start)))
    except:
        print('---Failed to retrieve---')
        print('url:', (url + str(start)))
        continue
    soup = BeautifulSoup(page.text, 'html.parser')
    # extract info from class:row
    # company name and job title
    companies, jobs = [], []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        company = div.find_all(name = 'span', attrs = {'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())
        for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
            jobs.append(a['title'])
            
    # extract location
    locations = []
    spans = soup.find_all('span', attrs={'class':'location'})
    for span in spans:
        locations.append(span.text)
        
    # extract summaries
    summaries = []
    divs = soup.find_all('div', attrs = {'class':'summary'})
    for i, div in enumerate(divs):
        summaries.append(div.text.strip())
        
    df_temp = zip(locations, companies, jobs, summaries)
    df_temp = pd.DataFrame(df_temp, columns = columns)
    df = df.append(df_temp).reset_index(drop = True)
    
df.to_csv('sample_data.csv', encoding='utf-8')

retriving url:  https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=0


TypeError: data argument can't be an iterator

In [12]:
max_jobs_to_fetch = 300
columns = ['location', 'company_name', 'job_title', 'salary', 'summary', 'full_info']
df = pd.DataFrame(columns = columns)

for start in range(0, max_jobs_to_fetch, 10):
    try:
#         url = 'https://www.indeed.co.uk/jobs?q=finance&l=London&radius=25&start=' + str(start)
        url = 'https://www.indeed.co.uk/jobs?q=Finance&l=London&radius=100&start=' + str(start)
        page = requests.get(url)
        print('retriving url: ', url)
    except:
        print('---Failed to retrieve---')
        print('url:', url)
        continue
    soup = BeautifulSoup(page.text, 'html.parser')
    time.sleep(1)
    ## metadata from mainpage
    
    # extract info from class:row
    # company name and job title
    companies, jobs = [], []
    for div in soup.find_all(name = 'div', attrs = {'class':'row'}):
        company = div.find_all(name = 'span', attrs = {'class':'company'})
        if len(company) > 0:
            for b in company:
                companies.append(b.text.strip())
            else:
                sec_try = div.find_all(name = 'span', attrs = {'class':'result-link-source'})
                for span in sec_try:
                    companies.append(span.text.strip())
        for a in div.find_all(name = 'a', attrs = {'data-tn-element':"jobTitle"}):
            jobs.append(a['title'])
            
    # extract location
    locations = []
    spans = soup.find_all('span', attrs={'class':'location'})
    for span in spans:
        locations.append(span.text)
        
    # extract summaries
    summaries = []
    divs = soup.find_all('div', attrs = {'class':'summary'})
    for i, div in enumerate(divs):
        summaries.append(div.text.strip())
    
#     ## crawl to subpages
#     descriptions = []
#     for adlink in soup.select('a[onmousedown*="return rclk(this,jobmap["]'):
#         suburl = "https://www.indeed.com" + adlink['href']
#         try:
#             subpage = requests.get(suburl)
#             subsoup = BeautifulSoup(subpage.text)
#         except:
#             print('--- Failed to retrieve sub-URL ---')
#             print('url: ', suburl)
#         # extract descriptions
#         for des in subsoup.select('div[class*="jobsearch-JobComponent-description"]'): 
#              descriptions.append(des.get_text())
            
    df_temp = zip(locations, companies, jobs, summaries, descriptions)
    df_temp = pd.DataFrame(df_temp, columns = columns)
    df = df.append(df_temp).reset_index(drop = True)

retriving url:  https://www.indeed.co.uk/jobs?q=Finance&l=London&radius=100&start=0


TypeError: data argument can't be an iterator

In [53]:
df.to_csv('sample_full.csv', encoding = 'utf-8')